In [2]:
import pandas as pd

# Загрузка файла с метками
labels_file = 'bnu.csv'
labels_df = pd.read_csv(labels_file)

# Просмотр первых строк таблицы
print(labels_df.head())


   label
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0


In [3]:
import numpy as np

# Загрузка данных из файлов .npy
data1 = np.load('bnu1.npy')
data2 = np.load('bnu2.npy')

# Проверка формата данных
print(f"Форма данных из bnu1.npy: {data1.shape}")
print(f"Форма данных из bnu2.npy: {data2.shape}")


Форма данных из bnu1.npy: (71, 240, 419)
Форма данных из bnu2.npy: (71, 240, 419)


In [4]:
# Объединение данных
all_data = np.concatenate([data1, data2], axis=0)  # Объединяем данные по оси участников (первое измерение)
print(f"Форма объединенных данных: {all_data.shape}")


# Предположим, что в метках есть столбец 'label', где 0 — это одно состояние (например, глаза закрыты), а 1 — другое (глаза открыты)
labels = labels_df['label'].values  # Преобразуем метки в numpy массив
print(f"Количество меток: {len(labels)}")

# Проверка: количество меток должно совпадать с количеством записей (участников) в данных
assert all_data.shape[0] == len(labels), "Количество участников и меток не совпадает!"

Форма объединенных данных: (142, 240, 419)
Количество меток: 142


In [5]:
# Нормализация данных по каждому признаку (региону мозга)
# Для каждого участника, для каждого временного ряда нормализуем по среднему и стандартному отклонению
all_data_normalized = (all_data - np.mean(all_data, axis=1, keepdims=True)) / np.std(all_data, axis=1, keepdims=True)


In [6]:
from sklearn.model_selection import train_test_split

# Разделение данных и меток на обучающую и тестовую выборки (80% обучение, 20% тест)
X_train, X_test, y_train, y_test = train_test_split(all_data_normalized, labels, test_size=0.2, random_state=42)

print(f"Форма X_train: {X_train.shape}, Форма y_train: {y_train.shape}")
print(f"Форма X_test: {X_test.shape}, Форма y_test: {y_test.shape}")


Форма X_train: (113, 240, 419), Форма y_train: (113,)
Форма X_test: (29, 240, 419), Форма y_test: (29,)


In [8]:
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [9]:
# 1. Преобразование данных в корреляционные матрицы
def create_correlation_matrices(X):
    matrices = []
    for i in range(X.shape[0]):  # Для каждого участника
        corr_matrix = np.corrcoef(X[i].T)  # Корреляционная матрица для временных рядов
        matrices.append(corr_matrix)
    return np.array(matrices)

# Создание корреляционных матриц для обучающих и тестовых данных
X_train_corr = create_correlation_matrices(X_train)
X_test_corr = create_correlation_matrices(X_test)

# 2. Преобразование корреляционных матриц в одномерные векторы
X_train_corr_flatten = X_train_corr.reshape(X_train_corr.shape[0], -1)
X_test_corr_flatten = X_test_corr.reshape(X_test_corr.shape[0], -1)

# 3. Нормализация корреляционных матриц
scaler = StandardScaler()
X_train_corr_normalized = scaler.fit_transform(X_train_corr_flatten)
X_test_corr_normalized = scaler.transform(X_test_corr_flatten)

# 4. Обучение модели SVM на нормализованных матрицах
svm_clf = SVC(kernel='linear', probability=True)  # Используем вероятностные предсказания
svm_clf.fit(X_train_corr_normalized, y_train)

# 5. Оценка точности на тестовых данных
y_pred = svm_clf.predict(X_test_corr_normalized)
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность SVM на корреляционных матрицах: {accuracy * 100:.2f}%')

# 6. Получение вероятностей предсказания (для вероятностной модели)
y_pred_prob = svm_clf.predict_proba(X_test_corr_normalized)[:, 1]  # Вероятность положительного класса

# 7. Сохранение результатов в CSV-файл
output_df = pd.DataFrame({
    'True_Label': y_test,  # Истинные метки
    'Predicted_Label': y_pred,  # Предсказанные метки
    'Predicted_Probability': y_pred_prob  # Вероятности для положительного класса
})

# Сохранение в CSV-файл
output_df.to_csv('svm_predictions.csv', index=False)

print("Файл svm_predictions.csv успешно создан!")

Точность SVM на корреляционных матрицах: 93.10%
Файл svm_predictions.csv успешно создан!
